In [1]:
import numpy as np
import pandas as pd

from multiprocessing import Pool

import galaxyfunctions as func

### mass density for the bulge:  
I will be using the equations found in https://doi.org/10.1111/j.1365-2966.2011.18564.x while modifying them to fit ngc 1300.  
the first equation is as follows:  
$$
\rho _{b} (R, z) = \frac{\rho _{b, 0}}{(1 + r'/r_{0})^{\alpha}}exp[-(r'/r_{cut})]
$$  
where  
$$
r' = \sqrt{R^{2} + (z/q)^{2}}
$$

other sources:  
http://adsabs.harvard.edu/pdf/1989ApJ...344..669E  
https://arxiv.org/ftp/astro-ph/papers/0502/0502573.pdf

In [2]:
#bulge constants
a = 2.3               #slopes of the profile
r0 = 8.26e-6          #scale radius
r_cut = 2.1           #truncation radius
rho_b0 = 9.82e12      #scale density
q = 0.1               #axial ratio

a1 = 1.8
r01 = 0.075
r_cut1 = 2.1
rho_b01 = 9.93e10 
q1 = 0.5

def rho_b(R, z):
    rp = np.sqrt(R**2 + (z/q)**2)
    return rho_b0 / (1 + rp / r0)**a * np.exp(-rp / r_cut)

def rho_bmw(R, z):
    rp = np.sqrt(R**2 + (z/q1)**2)
    return rho_b01 / (1 + rp / r01)**a1 * np.exp(-rp / r_cut1)

### mass density for disk:  
the equation i will be using is:  
$$
\rho _{d}(R, z) = \frac{M_{d}}{4 \pi R_{d}^2 z_{d}}e^{-R/R_{d}} sech ^{2} (z/z_{d})
$$  
  
which is from:  
https://www.aanda.org/articles/aa/pdf/2018/10/aa33509-18.pdf  

extra sources:  
https://sites.astro.caltech.edu/~george/ay21/eaa/eaa-diskgals.pdf

In [3]:
Md = 8.73e10     #disk mass
sig = 2.1e8
Rd = 5.59        #disk scale length
zd = 1           #disk scale height, an apporximation based on the milky way's thick disks, unable to find actual value

In [4]:
def rho_d(R, z):
    return (Md / (4*np.pi*(Rd**2) * zd)) * np.exp(-R/Rd) * (1/(np.cosh(z/zd))**2)

In [5]:
def rho_dmw(R,z):
    return (sig/(2*zd))*np.exp(-(abs(z)/zd)-(R/Rd))

### mass density of halo  
the equation i use is:  
$$
\rho_{h}(r) = \frac{\rho _{h, 0}}{\frac{r}{r_{h}}(1 + \frac{r}{r_{h}})^{2}}
$$  
sources:  
https://arxiv.org/pdf/1009.0376.pdf  
https://www.aanda.org/articles/aa/pdf/2018/10/aa33509-18.pdf

In [6]:
rho_h0 = 8.0e11
rh = 10.88

rho_h01 = 8.46e6
rh1 = 20.2

def rho_h(r):
    x = r / rh
    return rho_h0 / (x * (1+x)**2)

def rho_hmw(r):
    x = r / rh1
    return rho_h01 / (x * (1+x)**2)

### calculate mass inside sphere  
$$
M _{in}(r) = \sum _{i} (\int _{-\pi}^{\pi} \int _0^{2\pi} \int _0^r \rho_{i} (r', \theta, \phi) r'^2 \sin \theta dr'd\theta d\phi )
$$

In [7]:
def Mrb(r):
    R_list = np.linspace(1e-9, r, 1000)
    dR = R_list[1]-R_list[0]
    mass = 0
    for R in R_list:
        if r == R:
            continue
        else:
            zlim = np.sqrt(r**2 - R**2)
            z_list = np.linspace(-zlim, zlim, int(2*zlim/dR)+2)
            dz = z_list[1] - z_list[0]
            for z in z_list:
                ring = rho_b(R, z)*2*np.pi*R*dR*dz
                mass += ring
    return mass

def Mrb_mw(r):
    R_list = np.linspace(1e-9, r, 1000)
    dR = R_list[1]-R_list[0]
    mass = 0
    for R in R_list:
        if r == R:
            continue
        else:
            zlim = np.sqrt(r**2 - R**2)
            z_list = np.linspace(-zlim, zlim, int(2*zlim/dR)+2)
            dz = z_list[1] - z_list[0]
            for z in z_list:
                ring = rho_bmw(R, z)*2*np.pi*R*dR*dz
                mass += ring
    return mass

def Mrd(r):
    R_list = np.linspace(1e-9, r, 1000)
    dR = R_list[1] - R_list[0]
    mass = 0
    for R in R_list:
        if r == R:
            continue
        else:
            zlim = np.sqrt(r**2 - R**2)
            z_list = np.linspace(-zlim, zlim, int(2*zlim/dR) +2)
            dz = z_list[1] - z_list[0]
            for z in z_list:
                ring = rho_d(R, z)*2*np.pi*R*dR*dz
                mass += ring
    return mass

def Mrd_mw(r):
    R_list = np.linspace(1e-9, r, 1000)
    dR = R_list[1] - R_list[0]
    mass = 0
    for R in R_list:
        if r == R:
            continue
        else:
            zlim = np.sqrt(r**2 - R**2)
            z_list = np.linspace(-zlim, zlim, int(2*zlim/dR) +2)
            dz = z_list[1] - z_list[0]
            for z in z_list:
                ring = rho_dmw(R, z)*2*np.pi*R*dR*dz
                mass += ring
    return mass

def Mrh(r):
    R_list = np.linspace(1e-9, r, 1000)
    dR = R_list[1] - R_list[0]
    mass = 0
    for R in R_list:
        mass += 4*np.pi*R**2*rho_h(R)*dR
    return mass

def Mrh_mw(r):
    R_list = np.linspace(1e-9, r, 1000)
    dR = R_list[1] - R_list[0]
    mass = 0
    for R in R_list:
        mass += 4*np.pi*R**2*rho_hmw(R)*dR
    return mass

### mass lists for the bulge, disk, and halo

In [8]:
galaxy_size = 30
size_list = np.linspace(1e-6, galaxy_size, 1000)
with open('galaxy_size_list.npy', 'wb') as f:
    np.save(f, size_list)

In [ ]:
#bulge
pool = Pool()
bmass = pool.map(func.Mrb, size_list)
pool.close()
pool.join()

with open('bulge_mass_list.npy', 'wb') as f:
    np.save(f, bmass)

In [9]:
#bulge - milky way
pool = Pool()
bmass = pool.map(func.Mrb_mw, size_list)
pool.close()
pool.join()

with open('mwbulge_mass_list.npy', 'wb') as f:
    np.save(f, bmass)

In [ ]:
#disk - milky way
pool = Pool()
dmassmw = pool.map(func.Mrd_mw, size_list)
pool.close()
pool.join()

with open('mwdisk_mass_list.npy', 'wb') as f:
    np.save(f, dmassmw)

In [ ]:
#disk
pool = Pool()
dmass = pool.map(func.Mrd, size_list)
pool.close()
pool.join()

with open('disk_mass_list.npy', 'wb') as f:
    np.save(f, dmass)

In [ ]:
#halo
pool = Pool()
hmass = pool.map(func.Mrh, size_list)
pool.close()
pool.join()

with open('halo_mass_list.npy', 'wb') as f:
    np.save(f, hmass)

In [10]:
#halo - milky way
pool = Pool()
hmass = pool.map(func.Mrh_mw, size_list)
pool.close()
pool.join()

with open('mwhalo_mass_list.npy', 'wb') as f:
    np.save(f, hmass)